In [24]:
import numpy as np 
import pandas as pd

In [25]:
weather_code = pd.read_csv("weather_code.csv")

In [26]:
weather_code


,location_id,latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation
0,0,52.548330,13.407822,46,7200,Europe/Berlin,CEST
1,1,53.532513,9.980879,11,7200,Europe/Berlin,CEST
2,2,48.119507,11.550000,524,7200,Europe/Berlin,CEST
3,3,50.087870,8.653846,117,7200,Europe/Berlin,CEST
4,4,52.337433,14.500000,28,7200,Europe/Berlin,CEST


In [27]:
weather_code['Geo_location']= weather_code['latitude'].astype(str) + ',' + weather_code['longitude'].astype(str)

In [28]:
weather_code

,location_id,latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation,Geo_location
0,0,52.548330,13.407822,46,7200,Europe/Berlin,CEST,"52.54833,13.407822"
1,1,53.532513,9.980879,11,7200,Europe/Berlin,CEST,"53.532513,9.980879"
2,2,48.119507,11.550000,524,7200,Europe/Berlin,CEST,"48.119507,11.55"
3,3,50.087870,8.653846,117,7200,Europe/Berlin,CEST,"50.08787,8.653846"
4,4,52.337433,14.500000,28,7200,Europe/Berlin,CEST,"52.337433,14.5"


In [29]:
weather_code = pd.DataFrame({
    'location_id': [0,1,2,3,4],
    'latitude': [52.54833, 53.532513, 48.119507, 50.08787, 52.337433],
    'longitude': [13.407822, 9.980879, 11.55, 8.653846, 14.5],
    'elevation': [34, 45, 30, 28, 50],
    'utc_offset_seconds': [3600, 3600, 3600, 3600, 3600],
    'timezone': ['CET', 'CET', 'CET', 'CET', 'CET'],
    'timezone_abbreviation': ['CEST', 'CEST', 'CEST', 'CEST', 'CEST']
    
})

#Function to determine the place based on coordinates
def get_place(latitude, longitude):
    if latitude == 52.54833 and longitude == 13.407822:
        return 'Berlin'
    elif latitude == 53.532513 and longitude == 9.980879:
        return 'Hamburg'  # Assuming city for given coordinates
    elif latitude == 48.119507 and longitude == 11.55:
        return 'Munich'  # Assuming city for given coordinates
    elif latitude == 50.08787 and longitude == 8.653846:
        return 'Frankfurt'  # Assuming city for given coordinates
    elif latitude == 52.337433 and longitude == 14.5:
        return 'Berlin'  # Assuming city for given coordinates
    else:
        return 'Unknown'

# Add the 'place' column
weather_code['Place'] = weather_code.apply(lambda row: get_place(row['latitude'], row['longitude']), axis=1)

# Print the DataFrame to verify the changes
print(weather_code)

   location_id   latitude  longitude  elevation  utc_offset_seconds timezone  \
0            0  52.548330  13.407822         34                3600      CET   
1            1  53.532513   9.980879         45                3600      CET   
2            2  48.119507  11.550000         30                3600      CET   
3            3  50.087870   8.653846         28                3600      CET   
4            4  52.337433  14.500000         50                3600      CET   

  timezone_abbreviation      Place  
0                  CEST     Berlin  
1                  CEST    Hamburg  
2                  CEST     Munich  
3                  CEST  Frankfurt  
4                  CEST     Berlin  


In [30]:
import sqlite3
from sqlalchemy import create_engine

In [31]:
conn = sqlite3.connect('Weather.db')
cursor = conn.cursor()

In [32]:
cursor.execute('''CREATE TABLE IF NOT EXISTS weather_code
                  (location_id INTEGER PRIMARY KEY,
                   latitude FLOAT NOT NULL,
                   longitude FLOAT NOT NULL,
                   elevation INT,
                   utc_offset_seconds INT,
                   timezone VARCHAR(32),
                   timezone_abbreviation CHAR(4),
                   Place VARCHAR(24))
                ''')

In [33]:
conn.commit()

In [34]:
for index, row in weather_code.iterrows():
    cursor.execute('''INSERT INTO weather_code(
                        location_id,
                        latitude,
                        longitude,
                        elevation,
                        utc_offset_seconds,
                        timezone,
                        timezone_abbreviation,
                        
                        Place)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                   (row['location_id'],
                    row['latitude'],
                    row['longitude'],
                    row['elevation'],
                    row['utc_offset_seconds'],
                    row['timezone'],
                    row['timezone_abbreviation'],
                    row['Place']))
conn.commit()


In [35]:
cursor.execute('''SELECT * FROM weather_code''')
rows = cursor.fetchall()
for row in rows:
    print(row)
    

(0, 52.54833, 13.407822, 34, 3600, 'CET', 'CEST', 'Berlin')
(1, 53.532513, 9.980879, 45, 3600, 'CET', 'CEST', 'Hamburg')
(2, 48.119507, 11.55, 30, 3600, 'CET', 'CEST', 'Munich')
(3, 50.08787, 8.653846, 28, 3600, 'CET', 'CEST', 'Frankfurt')
(4, 52.337433, 14.5, 50, 3600, 'CET', 'CEST', 'Berlin')


In [36]:
weather = pd.read_csv("weather.csv")

In [42]:
print(weather.columns)

Index(['location_id', 'time', 'temperature_2m (°C)',
       'relative_humidity_2m (%)', 'apparent_temperature (°C)',
       'precipitation (mm)', 'weather_code (wmo code)',
       'wind_speed_10m (km/h)'],
      dtype='object')


In [39]:
conn = sqlite3.connect('Weather.db')
cursor = conn.cursor()

In [ ]:
test_data.to_sql('test_data', con=engine, index=False, if_exists='replace')